In [1]:
import json
import os 
import re 
import time

import cv2
from   matplotlib import pyplot as plt
import numpy as np
from   PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# matplotlib config
plt.rc('image', cmap='gray')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize='large', edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0')

In [2]:
IMG_DIR = '../../../img'
DATA = f'{IMG_DIR}/bounding_box'

In [14]:
def draw_bounding_box(
        img, ymin, xmin, ymax, xmax, color=(255, 0, 0), thickness=5):
    '''
    Adds a bounding box to an image.
    
    Args:
      image: a PIL.Image object.
      xmin, ymin, xmax, ymax: limits of bounding box.
      color: color to draw bounding box.
      thickness: line thickness.
    '''
    img_w, img_h = img.shape
    cv2.rectangle(img, 
                  (int(xmin), int(ymin)), 
                  (int(xmax), int(ymax)), 
                  color,
                  thickness)

In [15]:
def draw_bounding_boxes(img, boxes, color=[], thickness=5):
    '''
    Draws bounding boxes on image.
    
    Args:
      image: a PIL.Image object.
      boxes: a 2 dimensional numpy array of [N, 4]: 
        (xmin, ymin, xmax, ymax).
        The coordinates are in normalized format between [0, 1].
      color: color to draw bounding box
      thickness: line thickness
                           
    Raises:
      ValueError: if boxes is not a [N, 4] array
    '''
    boxes_shape = boxes.shape
    if not boxes_shape:
        return
    if len(boxes_shape) != 2 or boxes_shape[1] != 4:
        raise ValueError('boxes must be of size [N, 4]')
    for i in range(boxes_shpae[0]):
        draw_bounding_box(img, 
                          boxes[i, 1], 
                          boxes[i, 0],
                          boxes[i, 3],
                          boxes[i, 2],
                          color[i],
                          thickness)

In [5]:
def draw_bounding_boxes_on_img_array(img, boxes, color=[], thickness=5):
    '''
    Draws bounding boxes on image (numpy array).

    Args:
      image: a numpy array object.
      boxes: a 2 dimensional numpy array of [N, 4]: 
        (xmin, ymin, xmax, ymax).
        The coordinates are in normalized format between [0, 1].
      color: color to draw bounding box
      thickness: line thickness
    
    Raises:
      ValueError: if boxes is not a [N, 4] array
    '''
    draw_bounding_boxes(img, boxes, color, thickness)
    return img

In [8]:
def display_digits_with_boxes(
        imgs, pred_boxes, boxes, iou, title): #, boxes_normed=False):
    n = len(imgs)
    fig = plt.figure(figsize=(20, 4))
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    for i in range(n):
        ax = fig.add_subplot(1, 10, i + 1)
        boxes_to_plot = []
        if (len(pred_boxes) > i):
            box = pred_boxes[i]
            box = [bbox[0] * images[i].shape[1], 
                   bbox[1] * images[i].shape[0], 
                   bbox[2] * images[i].shape[1], 
                   bbox[3] * images[i].shape[0]]
            boxes_to_plot.append(box)
        img_to_draw = draw_bounding_boxes_on_img_array(
            img=imgs[i], 
            boxes=np.asarray(boxes_to_plot), 
            color=[(255, 0, 0), (0, 255, 0)])
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img_to_draw)
        if len(iou) > i:
            color = 'black'
            #if (iou[i][0] < iou_thresh):
            #    color = 'red'
            ax.text(0.2, 
                    -0.3, 
                    f'iou: {iou[i][0]}', 
                    color=color, 
                    transform=ax.transAxes)

In [9]:
def plot_metrics(history, metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(
        history.history[metric_name], color='blue', label=metric_name)
    val_name = f'val_{metric_name}'
    plt.plot(
        history.history[val_name], color='green', label=val_name)

# Load and Preprocess

In [10]:
def read_img_tfds(img, box):
    img = tf.cast(img, tf.float32)
    shape = tf.shape(img)
    x_factor = tf.cast(shape[1], tf.float32)
    y_factor = tf.cast(shape[0], tf.float32)
    img = tf.image.resize(img, (224, 224,))
    img /= 127.5
    img -= 1
    box_list = [box[0] / x_factor, 
                box[1] / y_factor, 
                box[2] / x_factor, 
                box[3] / y_factor]
    return img, box_list

In [11]:
def read_img_with_shape(img, box):
    orig_img = img
    img, box_list = read_img_tfds(img, box)
    return orig_img, img, box_list

In [13]:
def read_img_tfds_with_original_box(data):
    img = data['image']
    box = data['bbox']
    shape = tf.shape(image)
    x_factor, y_factor = [tf.cast(shape[d], tf.float32) for d in [1, 0]]
    box_list = [box[1] * x_factor,
                box[0] * y_factor,
                box[3] * x_factor,
                box[2] * y_factor]
    return img, box_list

In [ ]:
def dataset_to_numpy_util(dataset, batch_size=0, n=0):